# M12 Seminar: Economic and Social Problems: Insights from Big Data.
# Term Paper. 
# Replication file №3 - time series analysis

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

### Load US time series data
### Files can be found [here](https://github.com/social-connectedness-index/example-scripts)

In [2]:
ts_df = pd.read_csv('covid19_exploration/_intermediate/time_series_regress_dat.csv', encoding='cp1251')
ts_df['date'] = pd.to_datetime(ts_df['date'])
print(ts_df.shape)

(53363, 91)


In [3]:
# filter by date
ts_df_cases = ts_df.loc[(ts_df['date'] >= '2020-03-30') & (ts_df['date'] <= '2020-11-02')]
ts_df_deaths = ts_df.loc[(ts_df['date'] >= '2020-04-28') & (ts_df['date'] <= '2020-11-02')]

### Panel A - Social proximity to cases

In [5]:
# specify needed variables for panel A
y_cases = ts_df_cases[['log_chg_cases_10k']]

x_cases = ts_df_cases[['log_l1_chg_swc_10k', 
                 'log_l2_chg_swc_10k', 
                 'share_within50', 
                 'share_within150', 
                 'log_l1_chg_dwc_10k', 
                 'log_l2_chg_dwc_10k', 
                 'log_l1_chg_cases_10k', 
                 'log_l2_chg_cases_10k']] 

fixed_effects_raw = ts_df_cases[['week_num', 
                           'popdensity2010_10k', 
                           'med_hhinc2016', 
                           'state_FIPS']]

In [6]:
# make fixed effects

# ordinal encoder for fixed effects construction
encoder = preprocessing.OrdinalEncoder()

fixed_effects_raw['time_popdensity'] = fixed_effects_raw['week_num'] * fixed_effects_raw['popdensity2010_10k']
fixed_effects_raw['time_medhhinc'] = fixed_effects_raw['week_num'] * fixed_effects_raw['med_hhinc2016']
fixed_effects_raw['time_state'] = fixed_effects_raw['week_num'] * fixed_effects_raw['state_FIPS']

time_popdensity_fe = encoder.fit_transform(fixed_effects_raw['time_popdensity'].values.reshape(-1, 1))
time_medhhinc_fe = encoder.fit_transform(fixed_effects_raw['time_medhhinc'].values.reshape(-1, 1))
time_state_fe = encoder.fit_transform(fixed_effects_raw['time_state'].values.reshape(-1, 1))
# include FEs into DataFrames
x_cases['time_popdensity_fe'] = time_popdensity_fe
x_cases['time_medhhinc_fe'] = time_medhhinc_fe
x_cases['time_state_fe'] = time_state_fe

In [7]:
# function which runs OLS regression for cases (any specification)
def run_regression_cases(cols_to_use):
    x = x_cases[cols_to_use]
    x['target'] = y_cases
    #x['const'] = 1
    x = x.dropna()
    y = x['target']
    x = x.drop(['target'], axis=1)
    print(x.shape, y.shape)
    print(f'mean target = {np.round(np.mean(y), 3)}')
    results = sm.OLS(y, x).fit()
    print(results.summary2())
    return results

In [8]:
# 8 specifications
cols_cases_1 = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_cases_2 = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_cases_3 = ['share_within50', 'share_within150', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_cases_4 = ['share_within50', 'share_within150', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_cases_5 = ['log_l1_chg_dwc_10k', 'log_l2_chg_dwc_10k', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_cases_6 = ['log_l1_chg_dwc_10k', 'log_l2_chg_dwc_10k', 'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_cases_7 = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k', 
                'share_within50', 'share_within150',
                'log_l1_chg_dwc_10k', 'log_l2_chg_dwc_10k', 
                'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_cases_8 = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k', 
                'share_within50', 'share_within150',
                'log_l1_chg_dwc_10k', 'log_l2_chg_dwc_10k', 
                'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

In [9]:
# panel A specification - 1
print('specification A1')
res_a1 = run_regression_cases(cols_to_use=cols_cases_1)
print('X' * 100)
# panel A specification - 2
print('specification A2')
res_a2 = run_regression_cases(cols_to_use=cols_cases_2)
print('X' * 100)
# panel A specification - 3
print('specification A3')
res_a3 = run_regression_cases(cols_to_use=cols_cases_3)
print('X' * 100)
# panel A specification - 4
print('specification A4')
res_a4 = run_regression_cases(cols_to_use=cols_cases_4)
print('X' * 100)
# panel A specification - 5
print('specification A5')
res_a5 = run_regression_cases(cols_to_use=cols_cases_5)
print('X' * 100)
# panel A specification - 6
print('specification A6')
res_a6 = run_regression_cases(cols_to_use=cols_cases_6)
print('X' * 100)
# panel A specification - 7
print('specification A7')
res_a7 = run_regression_cases(cols_to_use=cols_cases_7)
print('X' * 100)
# panel A specification - 8
print('specification A8')
res_a8 = run_regression_cases(cols_to_use=cols_cases_8)

specification A1
(47085, 6) (47085,)
mean target = 2.178
                         Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.914      
Dependent Variable:     target           AIC:                         105411.9575
Date:                   2021-12-21 21:17 BIC:                         105464.5158
No. Observations:       47085            Log-Likelihood:              -52700.    
Df Model:               6                F-statistic:                 8.331e+04  
Df Residuals:           47079            Prob (F-statistic):          0.00       
R-squared (uncentered): 0.914            Scale:                       0.54923    
-----------------------------------------------------------------------------------
                          Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------------------
log_l1_chg_swc_10k        0.7504     0.0120    62.6165   0.0

                         Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.913      
Dependent Variable:     target           AIC:                         105728.3548
Date:                   2021-12-21 21:17 BIC:                         105789.6727
No. Observations:       47085            Log-Likelihood:              -52857.    
Df Model:               7                F-statistic:                 7.089e+04  
Df Residuals:           47078            Prob (F-statistic):          0.00       
R-squared (uncentered): 0.913            Scale:                       0.55292    
-----------------------------------------------------------------------------------
                          Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------------------
log_l1_chg_dwc_10k        0.3663     0.0064    57.2766   0.0000    0.3537    0.3788
log_l2_chg_dwc_10k       -0.2449 

### Panel B - Social proximity to deaths

In [11]:
# panel B specification - full
y_deaths = ts_df_deaths[['log_chg_deaths_10k_4wk']]

x_deaths = ts_df_deaths[['log_l2_chg_swd_10k_4wk', 
                 'log_l4_chg_swd_10k_4wk', 
                 'share_within50', 
                 'share_within150', 
                 'log_l2_chg_dwd_10k_4wk', 
                 'log_l4_chg_dwd_10k_4wk', 
                 'log_l2_chg_deaths_10k_4wk', 
                 'log_l4_chg_deaths_10k_4wk']] 

fixed_effects_raw = ts_df_deaths[['week_num', 
                           'popdensity2010_10k', 
                           'med_hhinc2016', 
                           'state_FIPS']]

# ordinal encoder for fixed effects construction
encoder = preprocessing.OrdinalEncoder()

fixed_effects_raw['time_popdensity'] = fixed_effects_raw['week_num'] * fixed_effects_raw['popdensity2010_10k']
fixed_effects_raw['time_medhhinc'] = fixed_effects_raw['week_num'] * fixed_effects_raw['med_hhinc2016']
fixed_effects_raw['time_state'] = fixed_effects_raw['week_num'] * fixed_effects_raw['state_FIPS']

time_popdensity_fe = encoder.fit_transform(fixed_effects_raw['time_popdensity'].values.reshape(-1, 1))
time_medhhinc_fe = encoder.fit_transform(fixed_effects_raw['time_medhhinc'].values.reshape(-1, 1))
time_state_fe = encoder.fit_transform(fixed_effects_raw['time_state'].values.reshape(-1, 1))
# add FEs to the main DataFrame
x_deaths['time_popdensity_fe'] = time_popdensity_fe
x_deaths['time_medhhinc_fe'] = time_medhhinc_fe
x_deaths['time_state_fe'] = time_state_fe

In [12]:
# function that fits OLS regression for deaths (any specification)
def run_regression_deaths(cols_to_use):
    x = x_deaths[cols_to_use]
    x['target'] = y_deaths
    #x['const'] = 1
    x = x.dropna()
    y = x['target']
    x = x.drop(['target'], axis=1)
    print(x.shape, y.shape)
    print(f'mean target = {np.round(np.mean(y), 3)}')
    results = sm.OLS(y, x).fit()
    print(results.summary2())
    return results

In [13]:
# 8 specifications for deaths
cols_deaths_1 = ['log_l2_chg_swd_10k_4wk', 'log_l4_chg_swd_10k_4wk', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_deaths_2 = ['log_l2_chg_swd_10k_4wk', 'log_l4_chg_swd_10k_4wk', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_deaths_3 = ['share_within50', 'share_within150', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_deaths_4 = ['share_within50', 'share_within150', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_deaths_5 = ['log_l2_chg_dwd_10k_4wk', 'log_l4_chg_dwd_10k_4wk',  
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_deaths_6 = ['log_l2_chg_dwd_10k_4wk', 'log_l4_chg_dwd_10k_4wk',  
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk',
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

cols_deaths_7 = ['log_l2_chg_swd_10k_4wk', 'log_l4_chg_swd_10k_4wk', 
                'share_within50', 'share_within150',
                'log_l2_chg_dwd_10k_4wk', 'log_l4_chg_dwd_10k_4wk', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk', 
                'time_popdensity_fe', 'time_medhhinc_fe']

cols_deaths_8 = ['log_l2_chg_swd_10k_4wk', 'log_l4_chg_swd_10k_4wk', 
                'share_within50', 'share_within150',
                'log_l2_chg_dwd_10k_4wk', 'log_l4_chg_dwd_10k_4wk', 
                'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk',  
                'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

In [14]:
# panel B specification - 1
print('specification B1')
res_b1 = run_regression_deaths(cols_to_use=cols_deaths_1)
print('X' * 100)
# panel B specification - 2
print('specification B2')
res_b2 = run_regression_deaths(cols_to_use=cols_deaths_2)
print('X' * 100)
# panel B specification - 3
print('specification B3')
res_b3 = run_regression_deaths(cols_to_use=cols_deaths_3)
print('X' * 100)
# panel B specification - 4
print('specification B4')
res_b4 = run_regression_deaths(cols_to_use=cols_deaths_4)
print('X' * 100)
# panel B specification - 5
print('specification B5')
res_b5 = run_regression_deaths(cols_to_use=cols_deaths_5)
print('X' * 100)
# panel B specification - 6
print('specification B6')
res_b6 = run_regression_deaths(cols_to_use=cols_deaths_6)
print('X' * 100)
# panel B specification - 7
print('specification B7')
res_b7 = run_regression_deaths(cols_to_use=cols_deaths_7)
print('X' * 100)
# panel B specification - 8
print('specification B8')
res_b8 = run_regression_deaths(cols_to_use=cols_deaths_8)

specification B1
(40807, 6) (40807,)
mean target = 0.369
                        Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.535     
Dependent Variable:     target           AIC:                         46628.5964
Date:                   2021-12-21 21:17 BIC:                         46680.2960
No. Observations:       40807            Log-Likelihood:              -23308.   
Df Model:               6                F-statistic:                 7819.     
Df Residuals:           40801            Prob (F-statistic):          0.00      
R-squared (uncentered): 0.535            Scale:                       0.18353   
---------------------------------------------------------------------------------
                               Coef.   Std.Err.     t     P>|t|    [0.025  0.975]
---------------------------------------------------------------------------------
log_l2_chg_swd_10k_4wk         0.5636    0.0149  37.8006  0.0000   0.5344 

                        Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.531     
Dependent Variable:     target           AIC:                         46952.1598
Date:                   2021-12-21 21:17 BIC:                         47012.4760
No. Observations:       40807            Log-Likelihood:              -23469.   
Df Model:               7                F-statistic:                 6603.     
Df Residuals:           40800            Prob (F-statistic):          0.00      
R-squared (uncentered): 0.531            Scale:                       0.18499   
---------------------------------------------------------------------------------
                              Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
---------------------------------------------------------------------------------
log_l2_chg_dwd_10k_4wk        0.1682    0.0053  31.5432  0.0000   0.1578   0.1787
log_l4_chg_dwd_10k_4wk       -0.0401    0.0053  -

### Now we need to construct 2 panels from 18 resulting OLS tables

In [15]:
# function that parces p-values in numeric format and returns p-values in ***/**/* format
def get_params_stars(ols_results):
    p_value_sign = []
    for i in ols_results.pvalues.values:
        if i < 0.01:
            p_value_sign.append('***')
        elif i < 0.05:
            p_value_sign.append('**')
        elif i < 0.1:
            p_value_sign.append('*')
        else:
            p_value_sign.append(' ')
    params_stars = []
    for i in range(ols_results.params.shape[0]):
        params_stars.append(str(ols_results.params.round(3).iloc[i]) + p_value_sign[i])
    output_df = pd.DataFrame()
    output_df.index = ols_results.params.index
    output_df['coef'] = params_stars
    return output_df

In [16]:
# replicate panel A
cases_res = pd.DataFrame()
cases_res.index = ['log_l1_chg_swc_10k', 'log_l2_chg_swc_10k', 
                   'share_within50', 'share_within150',
                   'log_l1_chg_dwc_10k', 'log_l2_chg_dwc_10k', 
                   'log_l1_chg_cases_10k', 'log_l2_chg_cases_10k', 
                   'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

for res in [res_a1, res_a2, res_a3, res_a4, res_a5, res_a6, res_a7, res_a8]:
    res_df = get_params_stars(res)
    cases_res = pd.merge(left=cases_res, right=res_df, how='left', left_index=True, right_index=True)
cases_res.columns = [f'({i})' for i in range(1, 9)]
cases_res = cases_res.fillna('-')

In [18]:
# print(cases_res.to_latex())

In [19]:
# replicate panel B
deaths_res = pd.DataFrame()
deaths_res.index = ['log_l2_chg_swd_10k_4wk', 'log_l4_chg_swd_10k_4wk', 
                    'share_within50', 'share_within150',
                    'log_l2_chg_dwd_10k_4wk', 'log_l4_chg_dwd_10k_4wk', 
                    'log_l2_chg_deaths_10k_4wk', 'log_l4_chg_deaths_10k_4wk',  
                    'time_popdensity_fe', 'time_medhhinc_fe', 'time_state_fe']

for res in [res_b1, res_b2, res_b3, res_b4, res_b5, res_b6, res_b7, res_b8]:
    res_df = get_params_stars(res)
    deaths_res = pd.merge(left=deaths_res, right=res_df, how='left', left_index=True, right_index=True)
deaths_res.columns = [f'({i})' for i in range(1, 9)]
deaths_res = deaths_res.fillna('-')

In [2]:
# print(deaths_res.to_latex())